In [1]:
!pip install -q imageio
!pip install -q opencv-python
!pip install -q git+https://github.com/tensorflow/docs
!pip install tensorflow



Defaulting to user installation because normal site-packages is not writeable


In [7]:
import os
import cv2
import ssl
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, MaxPooling3D, TimeDistributed, Dropout, Flatten, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import itertools
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm
import matplotlib.pyplot as plt
import tempfile
from urllib import request
import re


In [9]:
UCF_ROOT = "E:/2024t2/5925/A-Dataset-for-Automatic-Violence-Detection-in-Videos-master/renamed_videos"

def list_350_videos():
    """Lists videos available in the specified directory."""
    videos = [f for f in os.listdir(UCF_ROOT) if re.match(r"(non|v)_(?:[\w]+_){1,3}(cam1|cam2)_[0-9]+\.mp4", f)]
    return sorted(set(videos))


def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y:start_y+min_dim, start_x:start_x+min_dim]

def load_video(path, max_frames=20, resize=(224, 224)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]  # Convert BGR to RGB
            frames.append(frame)
            if max_frames != 0 and len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames) / 255.0

videos = list_350_videos()
print("Available videos:", videos)



Available videos: ['non_greet_cam1_13.mp4', 'non_greet_cam1_18.mp4', 'non_greet_cam1_19.mp4', 'non_greet_cam1_24.mp4', 'non_greet_cam1_30.mp4', 'non_greet_cam1_31.mp4', 'non_greet_cam1_34.mp4', 'non_greet_cam1_42.mp4', 'non_greet_cam1_43.mp4', 'non_greet_cam1_45.mp4', 'non_greet_cam1_46.mp4', 'non_greet_cam1_48.mp4', 'non_greet_cam1_52.mp4', 'non_greet_cam1_54.mp4', 'non_greet_cam1_55.mp4', 'non_greet_cam1_7.mp4', 'non_greet_cam1_8.mp4', 'non_greet_cam1_9.mp4', 'non_greet_cam2_13.mp4', 'non_greet_cam2_18.mp4', 'non_greet_cam2_19.mp4', 'non_greet_cam2_24.mp4', 'non_greet_cam2_30.mp4', 'non_greet_cam2_31.mp4', 'non_greet_cam2_34.mp4', 'non_greet_cam2_42.mp4', 'non_greet_cam2_43.mp4', 'non_greet_cam2_45.mp4', 'non_greet_cam2_46.mp4', 'non_greet_cam2_48.mp4', 'non_greet_cam2_52.mp4', 'non_greet_cam2_54.mp4', 'non_greet_cam2_55.mp4', 'non_greet_cam2_7.mp4', 'non_greet_cam2_8.mp4', 'non_greet_cam2_9.mp4', 'non_greet_handgestures_cam1_22.mp4', 'non_greet_handgestures_cam1_50.mp4', 'non_greet_

In [41]:
video_list = list_350_videos()[:280]

print(video_list)


['non_greet_cam1_13.mp4', 'non_greet_cam1_18.mp4', 'non_greet_cam1_19.mp4', 'non_greet_cam1_24.mp4', 'non_greet_cam1_30.mp4', 'non_greet_cam1_31.mp4', 'non_greet_cam1_34.mp4', 'non_greet_cam1_42.mp4', 'non_greet_cam1_43.mp4', 'non_greet_cam1_45.mp4', 'non_greet_cam1_46.mp4', 'non_greet_cam1_48.mp4', 'non_greet_cam1_52.mp4', 'non_greet_cam1_54.mp4', 'non_greet_cam1_55.mp4', 'non_greet_cam1_7.mp4', 'non_greet_cam1_8.mp4', 'non_greet_cam1_9.mp4', 'non_greet_cam2_13.mp4', 'non_greet_cam2_18.mp4', 'non_greet_cam2_19.mp4', 'non_greet_cam2_24.mp4', 'non_greet_cam2_30.mp4', 'non_greet_cam2_31.mp4', 'non_greet_cam2_34.mp4', 'non_greet_cam2_42.mp4', 'non_greet_cam2_43.mp4', 'non_greet_cam2_45.mp4', 'non_greet_cam2_46.mp4', 'non_greet_cam2_48.mp4', 'non_greet_cam2_52.mp4', 'non_greet_cam2_54.mp4', 'non_greet_cam2_55.mp4', 'non_greet_cam2_7.mp4', 'non_greet_cam2_8.mp4', 'non_greet_cam2_9.mp4', 'non_greet_handgestures_cam1_22.mp4', 'non_greet_handgestures_cam1_50.mp4', 'non_greet_handgestures_cam2_

In [42]:
import pandas as pd

labels = ['_'.join(video.split('_')[1:-2]) for video in video_list]

df = pd.DataFrame({'Video_Path': video_list, 'Label': labels})

print(df)



                  Video_Path      Label
0      non_greet_cam1_13.mp4      greet
1      non_greet_cam1_18.mp4      greet
2      non_greet_cam1_19.mp4      greet
3      non_greet_cam1_24.mp4      greet
4      non_greet_cam1_30.mp4      greet
..                       ...        ...
275       v_push_cam2_12.mp4       push
276  v_push_club_cam1_27.mp4  push_club
277  v_push_club_cam1_28.mp4  push_club
278  v_push_club_cam2_27.mp4  push_club
279  v_push_club_cam2_28.mp4  push_club

[280 rows x 2 columns]


In [43]:
IMAGE_HEIGHT,IMAGE_WIDTH = 224,224
SEQUENCE_LENGTH = 20

CLASSES_LIST=video_list

In [44]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D


In [46]:
def create_dataset():
  features = []
  labels = []
  video_files_paths = []
  base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3))
  
  for class_index, video_url in enumerate(CLASSES_LIST):
    print(f'Extracting Data of Class: {video_url}')

    video_path = os.path.join(UCF_ROOT, video_url)
    frames = load_video(video_path)


    if len(frames) == SEQUENCE_LENGTH:
      # Extract features using InceptionV3
      extracted_features = []
      for frame in frames:
        features_batch = np.expand_dims(frame, axis=0)  # Ensure single frame batch
        features_batch = base_model.predict(features_batch)
        extracted_features.append(features_batch)

      # Keep features as a sequence for LSTM (avoid flattening)
      features.append(extracted_features)
      labels.append(class_index)
      video_files_paths.append(video_path)

  features = np.asarray(features)  # Features remain a sequence of frame features
  labels = np.array(labels)

  return features, labels, video_files_paths


In [47]:
features,labels,video_files_paths = create_dataset()


Extracting Data of Class: non_greet_cam1_13.mp4
1/1 [==============================] - 0s 52ms/step
Extracting Data of Class: non_greet_cam1_18.mp4
1/1 [==============================] - 0s 55ms/step
Extracting Data of Class: non_greet_cam1_19.mp4
1/1 [==============================] - 0s 54ms/step
Extracting Data of Class: non_greet_cam1_24.mp4
1/1 [==============================] - 0s 53ms/step
Extracting Data of Class: non_greet_cam1_30.mp4
1/1 [==============================] - 0s 53ms/step
Extracting Data of Class: non_greet_cam1_31.mp4
1/1 [==============================] - 0s 49ms/step
Extracting Data of Class: non_greet_cam1_34.mp4
1/1 [==============================] - 0s 52ms/step
Extracting Data of Class: non_greet_cam1_42.mp4
1/1 [==============================] - 0s 51ms/step
Extracting Data of Class: non_greet_cam1_43.mp4
1/1 [==============================] - 0s 51ms/step
Extracting Data of Class: non_greet_cam1_45.mp4
1/1 [==============================] - 0s 53ms/step


In [48]:
df['Label']

0          greet
1          greet
2          greet
3          greet
4          greet
         ...    
275         push
276    push_club
277    push_club
278    push_club
279    push_club
Name: Label, Length: 280, dtype: object

In [49]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()


df['Label'] = label_encoder.fit_transform(df['Label'])

label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Mapping:", label_mapping)


Label Mapping: {'choke': 0, 'choke_fight': 1, 'choke_fight_push': 2, 'choke_punch_slap': 3, 'choke_slap': 4, 'choke_stab': 5, 'club': 6, 'club_fight': 7, 'club_fight_choke': 8, 'club_fight_kick': 9, 'club_fight_punch': 10, 'club_fight_push': 11, 'club_gunshot': 12, 'club_kick': 13, 'club_kick_fight': 14, 'club_slap': 15, 'club_stab': 16, 'fight': 17, 'fight_punch_push_kick': 18, 'fight_punch_slap': 19, 'greet': 20, 'greet_handgestures': 21, 'greet_handshake': 22, 'greet_highfive': 23, 'greet_walk': 24, 'gunshot': 25, 'gunshot_choke_stab': 26, 'gunshot_fight': 27, 'gunshot_stab': 28, 'handgestures': 29, 'handgestures_greet': 30, 'handgestures_highfive': 31, 'highfive': 32, 'highfive_greet': 33, 'hug': 34, 'hug_greet': 35, 'hug_greet_jump': 36, 'hug_highfive': 37, 'hug_jump': 38, 'jump': 39, 'jump_greet_hug': 40, 'jump_hug': 41, 'jump_punch_fight_kick': 42, 'kick': 43, 'kick_choke': 44, 'kick_fight': 45, 'punch': 46, 'punch_club_fight': 47, 'punch_fight': 48, 'punch_fight_kick': 49, 'pun

In [50]:
import random
import tensorflow as tf

y=df['Label']

x_train, x_test, y_train, y_test = train_test_split(features,y, test_size=0.2, random_state=42)


x_train.shape

(224, 20, 1, 5, 5, 2048)

In [51]:
NUM_CLASSES = len(np.unique(df['Label']))
print(NUM_CLASSES)

57


In [52]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten, TimeDistributed

# Define model architecture
model = Sequential()
model.add(TimeDistributed(Flatten(), input_shape=(x_train.shape[1:])))
model.add(LSTM(units=128))
model.add(Dropout(0.5))
model.add(Dense(NUM_CLASSES, activation='softmax'))
model.summary()


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_1 (TimeDi  (None, 20, 51200)         0         
 stributed)                                                      
                                                                 
 lstm_1 (LSTM)               (None, 128)               26280448  
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 57)                7353      
                                                                 
Total params: 26287801 (100.28 MB)
Trainable params: 26287801 (100.28 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [53]:
history = model.fit(x_train,y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)


Epoch 1/10
6/6 [==============================] - 23s 3s/step - loss: 4.3030 - accuracy: 0.0335 - val_loss: 4.0741 - val_accuracy: 0.0222
Epoch 2/10
6/6 [==============================] - 17s 3s/step - loss: 4.1201 - accuracy: 0.0335 - val_loss: 4.0337 - val_accuracy: 0.0444
Epoch 3/10
6/6 [==============================] - 17s 3s/step - loss: 4.0512 - accuracy: 0.0503 - val_loss: 3.9838 - val_accuracy: 0.0444
Epoch 4/10
6/6 [==============================] - 17s 3s/step - loss: 4.0057 - accuracy: 0.0223 - val_loss: 3.9828 - val_accuracy: 0.0222
Epoch 5/10
6/6 [==============================] - 17s 3s/step - loss: 3.8790 - accuracy: 0.0726 - val_loss: 3.9123 - val_accuracy: 0.0667
Epoch 6/10
6/6 [==============================] - 17s 3s/step - loss: 3.9195 - accuracy: 0.0391 - val_loss: 3.8481 - val_accuracy: 0.0889
Epoch 7/10
6/6 [==============================] - 17s 3s/step - loss: 3.8841 - accuracy: 0.0559 - val_loss: 3.8695 - val_accuracy: 0.0889
Epoch 8/10
6/6 [==================

In [54]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Predict on the testing set
y_pred = model.predict(x_test)

# Convert predictions to class labels
predicted_labels = np.argmax(y_pred, axis=1)

# Use y_test directly as true_labels
true_labels = y_test

# Calculate evaluation metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='weighted', zero_division=1)
recall = recall_score(true_labels, predicted_labels, average='weighted', zero_division=1)
f1 = f1_score(true_labels, predicted_labels, average='weighted')

print("Evaluation Metrics:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

conf_matrix = confusion_matrix(true_labels, predicted_labels)
print(conf_matrix)

2/2 [==============================] - 3s 1s/step
Evaluation Metrics:
Accuracy: 0.125
Precision: 0.71494708994709
Recall: 0.125
F1-Score: 0.03225806451612903
[[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 7 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0

In [58]:
def calculate_precision_per_class(conf_matrix):
    true_positives = np.diag(conf_matrix)
    predicted_positives = np.sum(conf_matrix, axis=0)
    precision_per_class = np.divide(true_positives, predicted_positives, out=np.zeros_like(true_positives, dtype=float), where=predicted_positives != 0)
    return precision_per_class

precision_per_class = calculate_precision_per_class(conf_matrix)
print("Precision per class:")
print(precision_per_class)


Precision per class:
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.12962963 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.        ]


In [2]:
import subprocess
def convert_notebook_to_html(notebook_path):
    try:
        subprocess.run(['jupyter', 'nbconvert', '--to', 'html', notebook_path], check=True)
        print(f"Successfully converted {notebook_path} to HTML.")
    except subprocess.CalledProcessError as e:
        print(f"Error converting {notebook_path}: {e}")

notebook_path = 'E:/2024t2/5925/A-Dataset-for-Automatic-Violence-Detection-in-Videos-master/AIRTlab.ipynb'
convert_notebook_to_html(notebook_path)


Successfully converted E:/2024t2/5925/A-Dataset-for-Automatic-Violence-Detection-in-Videos-master/AIRTlab.ipynb to HTML.
